# Εύρεση σημείων γύρω στην Κων/πολη

## Περιγραφή

Το notebook αυτό φτιάχνει ένα LLM και του ζητάει να βρει κάποια landmarks στην Κων/πολη και μετά για κάθε ένα από τα landmarks του ζητάει να βρει 10 landmarks γύρω από το πρώτο στα οποία θα μπορούσαν να τοποθετηθούν οχήματα. Κάθε ένα από τα γκρουπ των δεύτερων landmarks επιστρέφεται σαν geojson. 

## Χρήση

Αρχικά χρειαζόμαστε ένα API key. Αυτό για την Google μπορούμε να το βγάλουμε από [εδώ](https://aistudio.google.com/app/api-keys). Μετά δημιουργούμε ένα `.env` αρχείο στο φάκελο που βρίσκεται και αυτό το notebook και βάζουμε

```bash
GOOGLE_API_KEY=το-κλειδι-μας
```

Χρειάζεται επίσης να εγκαταστήσουμε τα requirements, τρέχοντας 

```bash
pip install -r requirements.txt
```

Μετά τρέχουμε το υπόλοιπο notebook. 

In [1]:
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv(find_dotenv())
chat = "GOOGLE"  # Αυτό καθορίζει το μοντέλο ανάμεσα στις επιλογές πιο κάτω

In [2]:
class CentralPOI(BaseModel):
    """A movie with details."""

    lat: float = Field(..., description="The latitude of the poi")
    lon: float = Field(..., description="The longitude of the poi")
    name: str = Field(..., description="The name of the POI")
    description: float = Field(
        ..., description="A short description of why this is a good POI"
    )


class CentralPOIResponse(BaseModel):
    pois: list[CentralPOI]


In [3]:
user_prompt = """
Please provide me in the following json format a list of 10 locations that would be ideal for deploying these vehicles in Istanbul:
Make sure to provide locations that are well-known, busy, and suitable for vehicle deployment.
Make sure to provide a variety of locations, including parks, shopping areas, historical landmarks and transportation hubs.
In description please provide a reasoning of why you think this is a good location.
"""

system_prompt = """
I am starting a new company and we will be deploying vehicles around the city of Istanbul. These vehicles 
include self-driving cars and scooters. 
For this reason, I need to create a list of locations which are more crowded and would be ideal to deploy
these vehicles. 
You are an expert in micro-mobility and urban planning in Istanbul.
"""


In [4]:
if chat == "GOOGLE":
    from langchain_google_genai import ChatGoogleGenerativeAI

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.1,
    )

if chat == "PPLX":
    from langchain_perplexity import ChatPerplexity

    llm = ChatPerplexity(temperature=0, model="sonar")

In [5]:
llm_structured = llm.with_structured_output(schema=CentralPOIResponse)

messages = [
    SystemMessage(system_prompt),
    HumanMessage(user_prompt),
]

response = llm_structured.invoke(messages)

In [6]:
print(response)

pois=[CentralPOI(lat=41.0136, lon=28.9897, name='Sultanahmet Square', description=1.01), CentralPOI(lat=41.0251, lon=28.9935, name='Grand Bazaar', description=1.02), CentralPOI(lat=41.037, lon=28.9754, name='Spice Bazaar', description=1.03), CentralPOI(lat=41.0307, lon=29.0141, name='Karaköy', description=1.04), CentralPOI(lat=41.0353, lon=28.9784, name='Galata Tower', description=1.05), CentralPOI(lat=41.0498, lon=29.0108, name='Taksim Square', description=1.06), CentralPOI(lat=41.0708, lon=29.0372, name='Bosphorus Bridge (European Side)', description=1.07), CentralPOI(lat=41.0276, lon=28.9784, name='Dolmabahçe Palace', description=1.08), CentralPOI(lat=41.0155, lon=28.9855, name='Eminönü Ferry Terminal', description=1.09), CentralPOI(lat=41.1094, lon=29.0249, name='Istinye Park', description=1.1)]


In [7]:
RADIUS = 2


def user_prompt_2(location):
    return f"""
For the specific location, please try to find notable places around it in a {RADIUS}km radius. 
These will be the places where the vehicles will be parked. Please provide up to 10 locations in this radius.
Finally please provide a geojson withe the above landmarks. 

LOCATION:
{location}
"""

In [8]:
class Feature(BaseModel):
    type: str
    properties: dict[str, str]
    geometry: dict[str, object]


class FeatureCollection(BaseModel):
    type: str
    features: list[Feature]

In [10]:
responses = []
llm_geojson = llm.with_structured_output(FeatureCollection)

for r in response.pois:
    messages2 = [
        SystemMessage(system_prompt),
        HumanMessage(user_prompt_2(r)),
    ]
    response2 = llm_geojson.invoke(messages2)
    print(response2.model_dump_json())
    responses.append(response2)

{"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Hagia Sophia Grand Mosque","description":"A historic architectural marvel."},"geometry":{"type":"Point","coordinates":[28.985,41.0085]}},{"type":"Feature","properties":{"name":"Topkapi Palace Museum","description":"Former residence of Ottoman sultans."},"geometry":{"type":"Point","coordinates":[28.9948,41.0131]}},{"type":"Feature","properties":{"name":"Basilica Cistern","description":"An ancient underground water reservoir."},"geometry":{"type":"Point","coordinates":[28.9801,41.009]}},{"type":"Feature","properties":{"name":"Grand Bazaar","description":"One of the largest and oldest covered markets in the world."},"geometry":{"type":"Point","coordinates":[28.9712,41.0122]}},{"type":"Feature","properties":{"name":"Spice Bazaar (Egyptian Bazaar)","description":"A colorful market filled with spices, sweets, and souvenirs."},"geometry":{"type":"Point","coordinates":[28.9744,41.0144]}},{"type":"Feature","properti

In [11]:
print(responses[0].model_dump_json())

{"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Hagia Sophia Grand Mosque","description":"A historic architectural marvel."},"geometry":{"type":"Point","coordinates":[28.985,41.0085]}},{"type":"Feature","properties":{"name":"Topkapi Palace Museum","description":"Former residence of Ottoman sultans."},"geometry":{"type":"Point","coordinates":[28.9948,41.0131]}},{"type":"Feature","properties":{"name":"Basilica Cistern","description":"An ancient underground water reservoir."},"geometry":{"type":"Point","coordinates":[28.9801,41.009]}},{"type":"Feature","properties":{"name":"Grand Bazaar","description":"One of the largest and oldest covered markets in the world."},"geometry":{"type":"Point","coordinates":[28.9712,41.0122]}},{"type":"Feature","properties":{"name":"Spice Bazaar (Egyptian Bazaar)","description":"A colorful market filled with spices, sweets, and souvenirs."},"geometry":{"type":"Point","coordinates":[28.9744,41.0144]}},{"type":"Feature","properti